In [1]:
from langchain_core.documents import Document 
from langchain_community.embeddings import OllamaEmbeddings 
from langchain_chroma import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate , MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.tools import tool , create_retriever_tool
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain.agents import create_tool_calling_agent , AgentExecutor
from langchain_core.messages import HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from pprint import pprint
import random

In [2]:
# Create documents
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [3]:
# Build vector store and retriever
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=OllamaEmbeddings(model="llama3")
)
retriever = vector_store.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 1}
)

rag_search = create_retriever_tool(
    retriever=retriever , 
    name="retrieve_animal_information" , 
    description="Search information about animal. For any question about animal, you must use this tool!"
)

@tool 
def multiply(a: int , b:int):
    """Multiply two numbers. When user ask about multiply two number, must use this function"""
    return a*b
print(multiply)

@tool 
def get_weather_temperature(location:str):
    """Get the current weather temperature in celcius"""
    return 19

@tool
def possible_area_where_cat_is_hiding() -> str:
    """Search for possible area where cat might be hiding"""
    return random.choice(["under the bed", "on the shelf" , "under table"])


@tool
def find_cat(place: str) -> str:
    """Use this tool to look up is the cat visible at that place"""
    if "bed" in place:  # For under the bed
        return "Cat not found"
    if "table" in place:  # For 'shelf'
        return "Found the cat"
    else:  # if the agent decides to ask about a different place
        return "Cat not found"

tools = [ rag_search , multiply , get_weather_temperature , possible_area_where_cat_is_hiding , find_cat]

name='multiply' description='Multiply two numbers. When user ask about multiply two number, must use this function' args_schema=<class 'pydantic.v1.main.multiplySchema'> func=<function multiply at 0x7fcfdff74b80>


In [4]:
# LLM model using open ai
llm = ChatOpenAI(
    model='gpt-3.5-turbo',
    api_key="API_KEY",
    #base_url="http://localhost:11434/v1"
) #.with_config({"tags": ["agent_llm"]})

In [5]:
prompt = ChatPromptTemplate.from_messages([
    ("ai", "you are helpfull assistance"),
    # MessagesPlaceholder("chat_history"),
    ("user" , "{input}") ,
    MessagesPlaceholder("agent_scratchpad")
])

In [8]:
agent = create_tool_calling_agent(llm , tools ,  prompt )
agent_exec = AgentExecutor(agent=agent , tools=tools, verbose=True) #.with_config({"run_name": "Agent"})

In [9]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='you are helpfull assistance')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')])
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fcfdff40290>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fcfdffb3190>, openai_api_ke

In [10]:
# get history function 
store = {}
def get_history(session_id:str) -> BaseChatMessageHistory: 
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    
    return store[session_id]

agent_exec_with_message_history = RunnableWithMessageHistory(
    runnable=agent_exec, 
    get_session_history=get_history , 
    input_messages_key='input', 
    history_messages_key='chat_history' ,
)


In [12]:
agent_exec_with_message_history 

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBranch(branches=[(RunnableBinding(bound=RunnableLambda(_is_not_async), config={'run_name': 'RunnableWithMessageHistoryInAsyncMode'}), RunnableBinding(bound=AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
    agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
  })
  | ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, me

In [11]:
chunks = []

for chunk in agent_exec_with_message_history.stream(
        {"input":"My cat is missing, please help me to find my cat until it found. Possible are is under the bed and under the table"} , {"configurable": {"session_id": "session_abc"}}
    ):
    # chunks.append(chunk)
    # print("---------")
    # pprint(chunk , depth=1)
    #pprint(chunk , depth=1)
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(f"Calling Tool: `{action.tool}` with input `{action.tool_input}`")
    # Observation
    elif "steps" in chunk:
        for step in chunk["steps"]:
            print(f"Tool Result: `{step.observation}`")
    # Final result
    elif "output" in chunk:
        print(f'Final Output: {chunk["output"]}')
    else:
        raise ValueError()
    print("---")



> Entering new AgentExecutor chain...
Calling Tool: `possible_area_where_cat_is_hiding` with input `{}`
---

Invoking: `possible_area_where_cat_is_hiding` with `{}`


under tableTool Result: `under table`
---
Calling Tool: `find_cat` with input `{'place': 'under table'}`
---

Invoking: `find_cat` with `{'place': 'under table'}`


Found the catTool Result: `Found the cat`
---
I found the cat under the table!

> Finished chain.
Final Output: I found the cat under the table!
---


In [40]:
chunks[2]['actions']

[ToolAgentAction(tool='get_items', tool_input={'place': 'shelf'}, log="\nInvoking: `get_items` with `{'place': 'shelf'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_Cnm6Hh9YPavgfJNVIEjyp00t', 'function': {'arguments': '{"place":"shelf"}', 'name': 'get_items'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-c0f2f50d-6776-4494-9522-ef8cd1d50024', tool_calls=[{'name': 'get_items', 'args': {'place': 'shelf'}, 'id': 'call_Cnm6Hh9YPavgfJNVIEjyp00t', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_items', 'args': '{"place":"shelf"}', 'id': 'call_Cnm6Hh9YPavgfJNVIEjyp00t', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_Cnm6Hh9YPavgfJNVIEjyp00t')]